In [1]:
import pickle
import time
from tqdm import tqdm
from datetime import timedelta, datetime
from dateutil import parser
import math
import sys
import pandas as pd
import json
import numpy as np

In [3]:
stored_vars = pickle.load(open('data/LINK_eth_transfer_list_final.pickle','rb'))

In [76]:
print(stored_vars["timeStampList"][1])
timeStampList = stored_vars["timeStampList"]
timeStampList1m = []
timeStampList1h = []
for i in range(len(timeStampList)):
    timeStampList1m.append(timeStampList[i][:17]+'00')
    timeStampList1h.append(timeStampList[i][:14]+'00:00')
print(timeStampList1m[0], timeStampList1h[0])

2021-11-30T16:35:07
2021-11-30T16:35:00 2021-11-30T16:00:00


In [73]:
temp_timestamp = datetime.fromisoformat(stored_vars["timeStampList"][1])
print(temp_timestamp)

2021-11-30 16:35:07


In [77]:
data = pd.DataFrame(list(zip(stored_vars["timeStampList"],stored_vars["transferValueList"],stored_vars["transferFromBalanceList"],stored_vars["transferToBalanceList"])), columns = ['time', 'volume', 'fromBalance', 'toBalance'])
data1h = pd.DataFrame(list(zip(timeStampList1h,stored_vars["transferValueList"],stored_vars["transferFromBalanceList"],stored_vars["transferToBalanceList"])), columns = ['time', 'volume', 'fromBalance', 'toBalance'])
data1m = pd.DataFrame(list(zip(timeStampList1m,stored_vars["transferValueList"],stored_vars["transferFromBalanceList"],stored_vars["transferToBalanceList"])), columns = ['time', 'volume', 'fromBalance', 'toBalance'])

In [78]:
data["time"] = pd.to_datetime(data["time"], infer_datetime_format=True)
data["volume"] = pd.to_numeric(data["volume"])
data["fromBalance"] = pd.to_numeric(data["fromBalance"])
data["toBalance"] = pd.to_numeric(data["toBalance"])

data1h["time"] = pd.to_datetime(data1h["time"], infer_datetime_format=True)
data1h["volume"] = pd.to_numeric(data1h["volume"])
data1h["fromBalance"] = pd.to_numeric(data1h["fromBalance"])
data1h["toBalance"] = pd.to_numeric(data1h["toBalance"])

data1m["time"] = pd.to_datetime(data1m["time"], infer_datetime_format=True)
data1m["volume"] = pd.to_numeric(data1m["volume"])
data1m["fromBalance"] = pd.to_numeric(data1m["fromBalance"])
data1m["toBalance"] = pd.to_numeric(data1m["toBalance"])

In [79]:
column_list = ["time", "meanVolume", "meanFromBalance", "meanToBalance", "medianVolume", "medianFromBalance",\
               "medianToBalance", "maxVolume", "maxFromBalance", "maxToBalance", "sumVolume", "sumFromBalance", "sumToBalance" ]

In [80]:
mean_data1h = data1h.groupby(["time"]).mean()
median_data1h = data1h.groupby(["time"]).median()
max_data1h = data1h.groupby(["time"]).max()
sum_data1h = data1h.groupby(["time"]).sum()

mean_data1m = data1m.groupby(["time"]).mean()
median_data1m = data1m.groupby(["time"]).median()
max_data1m = data1m.groupby(["time"]).max()
sum_data1m = data1m.groupby(["time"]).sum()

In [81]:
stat_data1h = pd.merge(mean_data1h, median_data1h, on="time")
stat_data1h = pd.merge(stat_data1h, max_data1h, on="time")
stat_data1h = pd.merge(stat_data1h, sum_data1h, on="time")
stat_data1h = pd.merge(pd.DataFrame(stat_data1h.index), stat_data1h, on="time")
stat_data1h.columns = column_list

stat_data1m = pd.merge(mean_data1m, median_data1m, on="time")
stat_data1m = pd.merge(stat_data1m, max_data1m, on="time")
stat_data1m = pd.merge(stat_data1m, sum_data1m, on="time")
stat_data1m = pd.merge(pd.DataFrame(stat_data1m.index), stat_data1m, on="time")
stat_data1m.columns = column_list

In [82]:
print(stat_data1m)

                     time    meanVolume  meanFromBalance  meanToBalance  \
0     2021-11-30 16:35:00  3.642499e+20     1.686824e+06    4791.565609   
1     2021-11-30 16:36:00  3.290219e+20     0.000000e+00  658049.550715   
2     2021-11-30 16:37:00  1.204266e+21     3.901061e+06    9583.131219   
3     2021-11-30 16:38:00  5.617914e+19     2.028188e+05  147058.317108   
4     2021-11-30 16:39:00  2.337520e+20     2.575410e+05      97.986140   
...                   ...           ...              ...            ...   
14869 2021-12-11 19:47:00  1.295969e+21     2.789912e+05   65283.437238   
14870 2021-12-11 19:49:00  2.583157e+19     1.492031e+06    1810.664895   
14871 2021-12-11 19:50:00  1.284027e+18     3.748392e+04       1.284027   
14872 2021-12-11 19:51:00  1.097508e+21     2.028490e+05  193793.580143   
14873 2021-12-11 19:52:00  9.749775e+18     0.000000e+00  178043.362814   

       medianVolume  medianFromBalance  medianToBalance     maxVolume  \
0      3.581353e+19       

In [83]:
print(stat_data1h)

                   time    meanVolume  meanFromBalance  meanToBalance  \
0   2021-11-30 16:00:00  8.214329e+20     6.006913e+05  133656.213149   
1   2021-11-30 17:00:00  8.043127e+20     4.407991e+05  521509.149010   
2   2021-11-30 18:00:00  1.100674e+21     4.106609e+05  881509.178126   
3   2021-11-30 19:00:00  1.098291e+21     6.098175e+05  584854.515687   
4   2021-11-30 20:00:00  9.155371e+20     6.148066e+05  490962.640229   
..                  ...           ...              ...            ...   
263 2021-12-11 15:00:00  1.517508e+21     1.249173e+06  458445.769676   
264 2021-12-11 16:00:00  2.889769e+21     7.073483e+05  446724.038937   
265 2021-12-11 17:00:00  1.032768e+21     4.535438e+05  756807.390443   
266 2021-12-11 18:00:00  2.581545e+20     3.858963e+05  302680.912414   
267 2021-12-11 19:00:00  1.299251e+21     6.890398e+05  486760.794633   

     medianVolume  medianFromBalance  medianToBalance     maxVolume  \
0    1.569324e+20         435.042246         7.08048